In [1]:
import os
import sys
import time
import random
import logging
import typing as tp
from pathlib import Path
from contextlib import contextmanager
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display
import numpy as np
import pandas as pd
from scipy import stats, special
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoost, Pool
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.compose import make_column_selector as selector
from sklearn import set_config
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from lightgbm import LGBMClassifier, LGBMRegressor
from scipy.stats import kurtosis, iqr, skew, gmean, hmean, kurtosistest, mode, normaltest, skewtest, shapiro, ks_2samp
from functools import partial
warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "widsdatathon2021"
WORK = ROOT / "working"

for path in DATA.iterdir():
    print(path.name)
    
train = pd.read_csv(DATA / "TrainingWiDS2021.csv")
test = pd.read_csv(DATA / "UnlabeledWiDS2021.csv")
sample_sub= pd.read_csv(DATA / "SampleSubmissionWiDS2021.csv")
print("train: {}, test: {}, sample sub: {}".format(
    train.shape, test.shape, sample_sub.shape
))

SampleSubmissionWiDS2021.csv
UnlabeledWiDS2021.csv
TrainingWiDS2021.csv
train: (130157, 181), test: (10234, 180), sample sub: (3, 2)


In [3]:
train.loc[train.age == 0, 'age'] = np.nan

In [4]:
test_id = test.encounter_id.values
train = train.drop(['readmission_status','encounter_id','hospital_id'], axis=1)
test = test.drop(['readmission_status','encounter_id','hospital_id'], axis=1)

In [5]:
train.head().T

,0,1,2,3,4
Unnamed: 0,1,2,3,4,5
age,68.000000,77.000000,25.000000,81.000000,19.0
bmi,22.732803,27.421875,31.952749,22.635548,NaN
elective_surgery,0,0,0,1,0
ethnicity,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
...,...,...,...,...,...
immunosuppression,0,0,0,0,0
leukemia,0,0,0,0,0
lymphoma,0,0,0,0,0
solid_tumor_with_metastasis,0,0,0,0,0


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(train.head(2))
    display(test.head(2))

,Unnamed: 0,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,2,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,NaN,NaN,159.0,67.0,159.0,67

,Unnamed: 0,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,1,72,NaN,0,Caucasian,F,152.4,Floor,Accident & Emergency,82,admit,Med-Surg ICU,0.015278,NaN,2.8,110.0,104.01,0,0,1.9,44.0,1.49,NaN,4.0,6.0,0.0,5.0,97.0,38.0,39.9,0,54.0,NaN,NaN,NaN,NaN,31.0,130.0,36.4,NaN,0,5.4,NaN,NaN,104.0,40.0,104.0,40.0,66.0,38.0,NaN,NaN,123.0,54.0,123.0,54.0,35.0,15.0,100.0,96.0,NaN,NaN,149.0,76.0,149.0,76.0,38.7,35.6,NaN,NaN,59.0,59.0,59.0,59.0,46.0,44.0,NaN,NaN,80.0,80.0,80.0,80.0,19.0,19.0,97.0,96.0,NaN,NaN,116.0,116.0,116.0,116.0,36.4,36.4,2.8,2.8,1.9,1.9,44.0,40.0,9.8,9.3,1.49,1.44,104.0,97.0,26.0,23.0,14.5,14.2,42.8,39.9,NaN,NaN,NaN,NaN,173.0,173.0,5.7,4.9,132.0,130.0,5.6,5.4,2.8,2.8,1.9,1.9,40.0,40.0,9.8,9.8,1.49,1.49,104.0,104.0,26.0,26.0,14.5,14.5,42.8,42.8,NaN,NaN,NaN,NaN,173.0,173.0,4.9,4.9,132.0,132.0,5.6,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1,2,86,NaN,0,Caucasian,F,175.3,Emergency Department,Accident & Emergency,82,admit,Med-Surg ICU,0.000000,NaN,NaN,117.0,106.01,0,0,NaN,19.0,0.92,NaN,4.0,6.0,0.0,5.0,73.0,116.0,NaN,0,41.0,NaN,NaN,NaN,NaN,53.0,142.0,36.3,NaN,0,NaN,NaN,NaN,101.0,27.0,101.0,27.0,116.0,56.0,NaN,NaN,129.0,41.0,129.0,41.0,51.0,14.0,100.0,95.0,NaN,Na

In [7]:
train = train.rename(columns={'pao2_apache':'pao2fio2ratio_apache','ph_apache':'arterial_ph_apache'})
test = test.rename(columns={'pao2_apache':'pao2fio2ratio_apache','ph_apache':'arterial_ph_apache'})

In [8]:
#Check Missing Data

print("Any missing sample in train set:",train.isnull().values.any(), "\n")


Any missing sample in train set: True 



In [9]:
## Print the categorical columns
print([c for c in train.columns if (1<train[c].nunique()) & (train[c].dtype != np.number)& (train[c].dtype != int) ])

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']


In [10]:

def ethnicity(x):
    if x=='Caucasian'              :   return 1
    if x=='African American'       :   return 2
    if x=='Other/Unknown'          :   return 3
    if x=='Hispanic'               :   return 4 
    if x=='Asian'                  :   return 5 


def gender(x):
    if x=='M'              :   return 1
    if x=='F'              :   return 2


def hospital_admit_source(x):
    if x=='Emergency Department'              :   return 1
    #if x=='NA'                               :   return 2
    if x=='Operating Room'                    :   return 2
    if x=='Floor'                             :   return 3 
    if x=='Direct Admit'                      :   return 4 

def icu_admit_source(x):
    if x=='Accident & Emergency'              :   return 1
    if x=='Operating Room / Recovery'         :   return 2
    if x=='Floor'                             :   return 3
    if x=='Other Hospital'                    :   return 4 
    if x=='Other ICU'                         :   return 5 


def icu_stay_type(x):
    if x=='admit'            :   return 1
    if x=='transfer'         :   return 2
    if x=='readmit'          :   return 3

def icu_type(x):
    if x=='Med-Surg ICU'      :   return 1
    if x=='CCU-CTICU'         :   return 2
    if x=='MICU'              :   return 3
    if x=='Neuro ICU'         :   return 4
    if x=='Cardiac ICU'       :   return 5






def categorize_age(x):
    
    if x<=35:
        return 1
    elif 35<x<=50:
        return 2
    elif 50<x<=65:
        return 3
    elif 65<x<=75:
        return 4
    elif x>75:
        return 5
    else:
        return 0

    
def categorize_bmi(x):
    
    if x<= 18.5:
        return 1
    elif 18.5<x<=25:
        return 2
    elif 25<x<=30:
        return 3
    elif 30<x<=35:
        return 4
    elif 35<x<=40:
        return 5
    elif x> 40:
        return 6
    else:
        return 0

def categorize_gcs_function(x):
    if x== 3:
        return 1
    elif 3<x<=6:
        return 2
    elif 6<x<=9:
        return 3
    elif 9<x<=12:
        return 4
    elif 12<x<=14:
        return 5
    elif x==15:
        return 6
    else:
        return 0

In [11]:
train['ethnicity'] = train['ethnicity'].apply(ethnicity)
train['gender'] = train['gender'].apply(gender)
train['hospital_admit_source'] = train['hospital_admit_source'].apply(hospital_admit_source)
train['icu_admit_source'] = train['icu_admit_source'].apply(icu_admit_source)
train['icu_stay_type'] = train['icu_stay_type'].apply(icu_stay_type)
train['icu_type'] = train['icu_type'].apply(icu_type)
train['cat_age'] = train['age'].apply(categorize_age)
train['cat_bmi'] = train['bmi'].apply(categorize_bmi)
test['ethnicity'] = test['ethnicity'].apply(ethnicity)
test['gender'] = test['gender'].apply(gender)
test['hospital_admit_source'] = test['hospital_admit_source'].apply(hospital_admit_source)
test['icu_admit_source'] = test['icu_admit_source'].apply(icu_admit_source)
test['icu_stay_type'] = test['icu_stay_type'].apply(icu_stay_type)
test['icu_type'] = test['icu_type'].apply(icu_type)
test['cat_age'] = test['age'].apply(categorize_age)
test['cat_bmi'] = test['bmi'].apply(categorize_bmi)
train['gcs_function'] = train['gcs_eyes_apache']+train['gcs_motor_apache']+train['gcs_verbal_apache']
test['gcs_function'] = test['gcs_eyes_apache']+test['gcs_motor_apache']+test['gcs_verbal_apache']
train['cat_gcs_function'] = train['gcs_function'].apply(categorize_gcs_function)
test['cat_gcs_function'] = test['gcs_function'].apply(categorize_gcs_function)

In [12]:
set(test.columns.tolist()) - set(['age',
  'bmi',
  'elective_surgery',
  'ethnicity',
  'gender',
  'hospital_admit_source',
  'icu_admit_source',
  'icu_id',
  'icu_stay_type',
  'icu_type',
  'pre_icu_los_days',
  'weight',
  'apache_2_diagnosis',
  'apache_3j_diagnosis',
  'arf_apache',
  'bilirubin_apache',
  'bun_apache',
  'creatinine_apache',
  'fio2_apache',
  'gcs_eyes_apache',
  'gcs_motor_apache',
  'gcs_verbal_apache',
  'glucose_apache',
  'heart_rate_apache',
  'intubated_apache',
  'sodium_apache',
  'temp_apache',
  'urineoutput_apache',
  'ventilated_apache',
  'd1_diasbp_invasive_max',
  'd1_diasbp_invasive_min',
  'd1_diasbp_noninvasive_max',
  'd1_diasbp_noninvasive_min',
  'd1_heartrate_max',
  'd1_heartrate_min',
  'd1_resprate_min',
  'd1_spo2_max',
  'd1_spo2_min',
  'd1_sysbp_invasive_max',
  'd1_sysbp_max',
  'd1_sysbp_min',
  'd1_sysbp_noninvasive_max',
  'd1_sysbp_noninvasive_min',
  'd1_temp_max',
  'h1_diasbp_invasive_max',
  'h1_diasbp_invasive_min',
  'h1_diasbp_max',
  'h1_diasbp_min',
  'h1_diasbp_noninvasive_min',
  'h1_sysbp_invasive_max',
  'h1_sysbp_invasive_min',
  'd1_albumin_max',
  'd1_bilirubin_max',
  'd1_bilirubin_min',
  'd1_bun_max',
  'd1_bun_min',
  'd1_calcium_min',
  'd1_creatinine_max',
  'd1_creatinine_min',
  'd1_glucose_max',
  'd1_glucose_min',
  'd1_hco3_max',
  'd1_hco3_min',
  'd1_hemaglobin_max',
  'd1_hemaglobin_min',
  'd1_hematocrit_max',
  'd1_inr_max',
  'd1_lactate_max',
  'd1_lactate_min',
  'd1_platelets_min',
  'd1_potassium_max',
  'd1_potassium_min',
  'd1_sodium_max',
  'd1_sodium_min',
  'd1_wbc_max',
  'h1_bilirubin_min',
  'h1_bun_min',
  'h1_calcium_max',
  'h1_calcium_min',
  'h1_creatinine_max',
  'h1_glucose_max',
  'h1_glucose_min',
  'h1_hco3_max',
  'h1_hco3_min',
  'h1_hemaglobin_max',
  'h1_inr_max',
  'h1_inr_min',
  'h1_potassium_max',
  'h1_potassium_min',
  'h1_sodium_max',
  'h1_sodium_min',
  'd1_arterial_pco2_max',
  'd1_arterial_ph_max',
  'd1_arterial_po2_max',
  'd1_arterial_po2_min',
  'h1_arterial_ph_max',
  'h1_arterial_ph_min',
  'h1_arterial_po2_min',
  'cirrhosis',
  'hepatic_failure',
  'immunosuppression',
  'leukemia',
  'solid_tumor_with_metastasis'])

{'Unnamed: 0',
 'aids',
 'albumin_apache',
 'apache_post_operative',
 'arterial_ph_apache',
 'cat_age',
 'cat_bmi',
 'cat_gcs_function',
 'd1_albumin_min',
 'd1_arterial_pco2_min',
 'd1_arterial_ph_min',
 'd1_calcium_max',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_hematocrit_min',
 'd1_inr_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_pao2fio2ratio_max',
 'd1_pao2fio2ratio_min',
 'd1_platelets_max',
 'd1_resprate_max',
 'd1_sysbp_invasive_min',
 'd1_temp_min',
 'd1_wbc_min',
 'gcs_function',
 'gcs_unable_apache',
 'h1_albumin_max',
 'h1_albumin_min',
 'h1_arterial_pco2_max',
 'h1_arterial_pco2_min',
 'h1_arterial_po2_max',
 'h1_bilirubin_max',
 'h1_bun_max',
 'h1_creatinine_min',
 'h1_diasbp_noninvasive_max',
 'h1_heartrate_max',
 'h1_heartrate_min',
 'h1_hemaglobin_min',
 'h1_hematocrit_max',
 'h1_hematocrit_min',
 'h1_lactate_max',
 'h1_lactate_min',
 'h1_mbp_invasive_max',
 'h1_mbp_invasi

In [13]:
df = pd.concat([train['icu_id'], test['icu_id']])

agg = df.value_counts().to_dict()
train['icu_id_counts'] = np.log1p(train['icu_id'].map(agg))
test['icu_id_counts'] = np.log1p(test['icu_id'].map(agg))

df = pd.concat([train['age'], test['age']])

agg = df.value_counts().to_dict()
train['age_counts'] = np.log1p(train['age'].map(agg))
test['age_counts'] = np.log1p(test['age'].map(agg))

In [14]:
train["diff_bmi"] = train['bmi'].copy() # orig BMI values
train['bmi'] = train['weight']/((train['height']/100)**2)
train["diff_bmi"] = train["diff_bmi"]-train['bmi']

test["diff_bmi"] = test['bmi'].copy() # orig BMI values
test['bmi'] = test['weight']/((test['height']/100)**2)
test["diff_bmi"] = test["diff_bmi"]-test['bmi']

In [15]:
d_cols = [c for c in train.columns if(c.startswith("d1"))]
h_cols = [c for c in train.columns if(c.startswith("h1"))]

train["dailyLabs_row_nan_count"] = train[d_cols].isna().sum(axis=1)
train["hourlyLabs_row_nan_count"] = train[h_cols].isna().sum(axis=1)
train
train["diff_labTestsRun_daily_hourly"] = train["dailyLabs_row_nan_count"] - train["hourlyLabs_row_nan_count"]

test["dailyLabs_row_nan_count"] = test[d_cols].isna().sum(axis=1)
test["hourlyLabs_row_nan_count"] = test[h_cols].isna().sum(axis=1)

test["diff_labTestsRun_daily_hourly"] = test["dailyLabs_row_nan_count"] - test["hourlyLabs_row_nan_count"]

In [16]:
min_max_feats=[f[:-4] for f in train.columns if f[-4:]=='_min']
for col in min_max_feats:
    train.loc[train[f'{col}_min'] > train[f'{col}_max'], [f'{col}_min', f'{col}_max']] = train.loc[train[f'{col}_min'] > train[f'{col}_max'], [f'{col}_max', f'{col}_min']].values
    test.loc[test[f'{col}_min'] > test[f'{col}_max'], [f'{col}_min', f'{col}_max']] = test.loc[test[f'{col}_min'] > test[f'{col}_max'], [f'{col}_max', f'{col}_min']].values
    print((train[f'{col}_min'] > train[f'{col}_max']).sum())
    print((test[f'{col}_min'] > test[f'{col}_max']).sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [17]:
lab_col = [c for c in train.columns if((c.startswith("h1")) | (c.startswith("d1")))]

# prefix (d1_, h1_) and Suffix (_min, max) removal from String list using map() + lambda 
lab_col_names = list(set(list(map(lambda i: i[ 3 : -4], lab_col))))

print("len lab_col",len(lab_col))
print("len lab_col_names",len(lab_col_names))
print("lab_col_names\n",lab_col_names)



len lab_col 128
len lab_col_names 32
lab_col_names
 ['wbc', 'creatinine', 'sysbp_invasive', 'hematocrit', 'arterial_ph', 'potassium', 'spo2', 'arterial_po2', 'inr', 'temp', 'diasbp', 'heartrate', 'sodium', 'resprate', 'mbp_noninvasive', 'hemaglobin', 'diasbp_noninvasive', 'sysbp_noninvasive', 'pao2fio2ratio', 'arterial_pco2', 'bilirubin', 'mbp', 'mbp_invasive', 'diasbp_invasive', 'sysbp', 'lactate', 'glucose', 'platelets', 'calcium', 'bun', 'hco3', 'albumin']


In [18]:
grouper = 'apache_3j_diagnosis'

first_h = []
for v in lab_col_names:
    colsx = [x for x in test.columns if v in x]
    train[v+"_nans"] = train.loc[:, colsx].isna().sum(axis=1)
    test[v+"_nans"] = test.loc[:, colsx].isna().sum(axis=1)

    train[v+"_d1_value_range"] = train[f"d1_{v}_max"].subtract(train[f"d1_{v}_min"])
    
    train[v+"_h1_value_range"] = train[f"h1_{v}_max"].subtract(train[f"h1_{v}_min"])
#     df[v+"_h1_value_range_normalized"] = df[f"h1_{v}_max"].subtract(df[f"h1_{v}_min"]).div(df[f"h1_{v}_max"])
    train[v+"_d1_h1_max_eq"] = (train[f"d1_{v}_max"]== train[f"h1_{v}_max"]).astype(np.int8)
    train[v+"_d1_h1_min_eq"] = (train[f"d1_{v}_min"]== train[f"h1_{v}_min"]).astype(np.int8)
    train[v+"_d1_zero_range"] = (train[v+"_d1_value_range"] == 0).astype(np.int8)
    train[v+"_h1_zero_range"] =(train[v+"_h1_value_range"] == 0).astype(np.int8)
    train['apache_3j_diagnosis_x'] = train['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
    train['apache_2_diagnosis_x'] = train['apache_2_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
    temp = pd.concat([train[[f"d1_{v}_max",grouper]], test[[f"d1_{v}_max",grouper]]], axis=0).groupby(grouper)[f"d1_{v}_max"].mean().to_dict()
    train[f'mean_diff_d1_{v}_max'] = train[f"d1_{v}_max"]-train[grouper].map(temp)
    test[f'mean_diff_d1_{v}_max'] = test[f"d1_{v}_max"]-test[grouper].map(temp)

    temp = pd.concat([train[[f"d1_{v}_min",grouper]], test[[f"d1_{v}_min",grouper]]], axis=0).groupby(grouper)[f"d1_{v}_min"].mean().to_dict()   
    train[f'mean_diff_d1_{v}_min'] = train[f"d1_{v}_min"]-train[grouper].map(temp)
    test[f'mean_diff_d1_{v}_min'] = test[f"d1_{v}_min"]-test[grouper].map(temp)
    temp = pd.concat([train[[f"h1_{v}_max",grouper]], test[[f"h1_{v}_max",grouper]]], axis=0).groupby(grouper)[f"h1_{v}_max"].mean().to_dict()   
    train[f'mean_diff_h1_{v}_max'] = train[f"h1_{v}_max"]-train[grouper].map(temp)
    test[f'mean_diff_h1_{v}_max'] = test[f"h1_{v}_max"]-test[grouper].map(temp)
    temp = pd.concat([train[[f"h1_{v}_min",grouper]], test[[f"h1_{v}_min",grouper]]], axis=0).groupby(grouper)[f"h1_{v}_min"].mean().to_dict()   
    
    train[f'mean_diff_h1_{v}_min'] = train[f"h1_{v}_min"]-train[grouper].map(temp)
    test[f'mean_diff_h1_{v}_min'] = test[f"h1_{v}_min"]-test[grouper].map(temp)

    # daily change in value range - hour vs day. could do subtract or div here.. 
    train[v+"_tot_change_value_range_normed"] = abs((train[v+"_d1_value_range"].div(train[v+"_h1_value_range"])))#.div(df[f"d1_{v}_max"]))
    
    # Cases where there's no reading in the first hour, but only  later in day ?
    train[v+"_started_after_firstHour"] = ((train[f"h1_{v}_max"].isna()) & (train[f"h1_{v}_min"].isna())) & (~train[f"d1_{v}_max"].isna())
    first_h.append(v+"_started_after_firstHour")
    
    ## Did a reading get more extreme after the first hour. 
    ## This misses cases where the readying is in an unhealthy region and "improves"!!
    train[v+"_day_more_extreme"] = ((train[f"d1_{v}_max"]>train[f"h1_{v}_max"]) | (train[f"d1_{v}_min"]<train[f"h1_{v}_min"]))
    train[v+"_day_more_extreme"].fillna(False)
    
    test[v+"_d1_value_range"] = test[f"d1_{v}_max"].subtract(test[f"d1_{v}_min"])
    
    test[v+"_h1_value_range"] = test[f"h1_{v}_max"].subtract(test[f"h1_{v}_min"])
#     df[v+"_h1_value_range_normalized"] = df[f"h1_{v}_max"].subtract(df[f"h1_{v}_min"]).div(df[f"h1_{v}_max"])
    test[v+"_d1_h1_max_eq"] = (test[f"d1_{v}_max"]== test[f"h1_{v}_max"]).astype(np.int8)
    test[v+"_d1_h1_min_eq"] = (test[f"d1_{v}_min"]== test[f"h1_{v}_min"]).astype(np.int8)
    test[v+"_d1_zero_range"] = (test[v+"_d1_value_range"] == 0).astype(np.int8)
    test[v+"_h1_zero_range"] =(test[v+"_h1_value_range"] == 0).astype(np.int8)
    test['apache_3j_diagnosis_x'] = test['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
    test['apache_2_diagnosis_x'] = test['apache_2_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
    # daily change in value range - hour vs day. could do subtract or div here.. 
    test[v+"_tot_change_value_range_normed"] = abs((test[v+"_d1_value_range"].div(test[v+"_h1_value_range"])))#.div(df[f"d1_{v}_max"]))
    
    # Cases where there's no reading in the first hour, but only  later in day ?
    test[v+"_started_after_firstHour"] = ((test[f"h1_{v}_max"].isna()) & (test[f"h1_{v}_min"].isna())) & (~test[f"d1_{v}_max"].isna())
    first_h.append(v+"_started_after_firstHour")
    
    ## Did a reading get more extreme after the first hour. 
    ## This misses cases where the readying is in an unhealthy region and "improves"!!
    test[v+"_day_more_extreme"] = ((test[f"d1_{v}_max"]>test[f"h1_{v}_max"]) | (test[f"d1_{v}_min"]<test[f"h1_{v}_min"]))
    test[v+"_day_more_extreme"].fillna(False)

train["total_Tests_started_After_firstHour"] = train[first_h].sum(axis=1)

test["total_Tests_started_After_firstHour"] = test[first_h].sum(axis=1)

train["total_Tests_started_After_firstHour"].describe()

count    130157.000000
mean         19.547992
std          11.493976
min           0.000000
25%          10.000000
50%          22.000000
75%          26.000000
max          62.000000
Name: total_Tests_started_After_firstHour, dtype: float64

In [19]:
train['diasbp_indicator'] = (
(train['d1_diasbp_invasive_max'] == train['d1_diasbp_max']) & (train['d1_diasbp_noninvasive_max']==train['d1_diasbp_invasive_max'])|
(train['d1_diasbp_invasive_min'] == train['d1_diasbp_min']) & (train['d1_diasbp_noninvasive_min']==train['d1_diasbp_invasive_min'])|
(train['h1_diasbp_invasive_max'] == train['h1_diasbp_max']) & (train['h1_diasbp_noninvasive_max']==train['h1_diasbp_invasive_max'])|
(train['h1_diasbp_invasive_min'] == train['h1_diasbp_min']) & (train['h1_diasbp_noninvasive_min']==train['h1_diasbp_invasive_min'])
).astype(np.int8)


train['mbp_indicator'] = (
(train['d1_mbp_invasive_max'] == train['d1_mbp_max']) & (train['d1_mbp_noninvasive_max']==train['d1_mbp_invasive_max'])|
(train['d1_mbp_invasive_min'] == train['d1_mbp_min']) & (train['d1_mbp_noninvasive_min']==train['d1_mbp_invasive_min'])|
(train['h1_mbp_invasive_max'] == train['h1_mbp_max']) & (train['h1_mbp_noninvasive_max']==train['h1_mbp_invasive_max'])|
(train['h1_mbp_invasive_min'] == train['h1_mbp_min']) & (train['h1_mbp_noninvasive_min']==train['h1_mbp_invasive_min'])
).astype(np.int8)

train['sysbp_indicator'] = (
(train['d1_sysbp_invasive_max'] == train['d1_sysbp_max']) & (train['d1_sysbp_noninvasive_max']==train['d1_sysbp_invasive_max'])|
(train['d1_sysbp_invasive_min'] == train['d1_sysbp_min']) & (train['d1_sysbp_noninvasive_min']==train['d1_sysbp_invasive_min'])|
 (train['h1_sysbp_invasive_max'] == train['h1_sysbp_max']) & (train['h1_sysbp_noninvasive_max']==train['h1_sysbp_invasive_max'])|
(train['h1_sysbp_invasive_min'] == train['h1_sysbp_min']) & (train['h1_sysbp_noninvasive_min']==train['h1_sysbp_invasive_min'])   
).astype(np.int8)

train['d1_mbp_invnoninv_max_diff'] = train['d1_mbp_invasive_max'] - train['d1_mbp_noninvasive_max']
train['h1_mbp_invnoninv_max_diff'] = train['h1_mbp_invasive_max'] - train['h1_mbp_noninvasive_max']
train['d1_mbp_invnoninv_min_diff'] = train['d1_mbp_invasive_min'] - train['d1_mbp_noninvasive_min']
train['h1_mbp_invnoninv_min_diff'] = train['h1_mbp_invasive_min'] - train['h1_mbp_noninvasive_min']
train['d1_diasbp_invnoninv_max_diff'] = train['d1_diasbp_invasive_max'] - train['d1_diasbp_noninvasive_max']
train['h1_diasbp_invnoninv_max_diff'] = train['h1_diasbp_invasive_max'] - train['h1_diasbp_noninvasive_max']
train['d1_diasbp_invnoninv_min_diff'] = train['d1_diasbp_invasive_min'] - train['d1_diasbp_noninvasive_min']
train['h1_diasbp_invnoninv_min_diff'] = train['h1_diasbp_invasive_min'] - train['h1_diasbp_noninvasive_min']

train['d1_sysbp_invnoninv_max_diff'] = train['d1_sysbp_invasive_max'] - train['d1_sysbp_noninvasive_max']
train['h1_sysbp_invnoninv_max_diff'] = train['h1_sysbp_invasive_max'] - train['h1_sysbp_noninvasive_max']
train['d1_sysbp_invnoninv_min_diff'] = train['d1_sysbp_invasive_min'] - train['d1_sysbp_noninvasive_min']
train['h1_sysbp_invnoninv_min_diff'] = train['h1_sysbp_invasive_min'] - train['h1_sysbp_noninvasive_min']

test['diasbp_indicator'] = (
(test['d1_diasbp_invasive_max'] == test['d1_diasbp_max']) & (test['d1_diasbp_noninvasive_max']==test['d1_diasbp_invasive_max'])|
(test['d1_diasbp_invasive_min'] == test['d1_diasbp_min']) & (test['d1_diasbp_noninvasive_min']==test['d1_diasbp_invasive_min'])|
(test['h1_diasbp_invasive_max'] == test['h1_diasbp_max']) & (test['h1_diasbp_noninvasive_max']==test['h1_diasbp_invasive_max'])|
(test['h1_diasbp_invasive_min'] == test['h1_diasbp_min']) & (test['h1_diasbp_noninvasive_min']==test['h1_diasbp_invasive_min'])
).astype(np.int8)


test['mbp_indicator'] = (
(test['d1_mbp_invasive_max'] == test['d1_mbp_max']) & (test['d1_mbp_noninvasive_max']==test['d1_mbp_invasive_max'])|
(test['d1_mbp_invasive_min'] == test['d1_mbp_min']) & (test['d1_mbp_noninvasive_min']==test['d1_mbp_invasive_min'])|
(test['h1_mbp_invasive_max'] == test['h1_mbp_max']) & (test['h1_mbp_noninvasive_max']==test['h1_mbp_invasive_max'])|
(test['h1_mbp_invasive_min'] == test['h1_mbp_min']) & (test['h1_mbp_noninvasive_min']==test['h1_mbp_invasive_min'])
).astype(np.int8)

test['sysbp_indicator'] = (
(test['d1_sysbp_invasive_max'] == test['d1_sysbp_max']) & (test['d1_sysbp_noninvasive_max']==test['d1_sysbp_invasive_max'])|
(test['d1_sysbp_invasive_min'] == test['d1_sysbp_min']) & (test['d1_sysbp_noninvasive_min']==test['d1_sysbp_invasive_min'])|
 (test['h1_sysbp_invasive_max'] == test['h1_sysbp_max']) & (test['h1_sysbp_noninvasive_max']==test['h1_sysbp_invasive_max'])|
(test['h1_sysbp_invasive_min'] == test['h1_sysbp_min']) & (test['h1_sysbp_noninvasive_min']==test['h1_sysbp_invasive_min'])   
).astype(np.int8)

test['d1_mbp_invnoninv_max_diff'] = test['d1_mbp_invasive_max'] - test['d1_mbp_noninvasive_max']
test['h1_mbp_invnoninv_max_diff'] = test['h1_mbp_invasive_max'] - test['h1_mbp_noninvasive_max']
test['d1_mbp_invnoninv_min_diff'] = test['d1_mbp_invasive_min'] - test['d1_mbp_noninvasive_min']
test['h1_mbp_invnoninv_min_diff'] = test['h1_mbp_invasive_min'] - test['h1_mbp_noninvasive_min']
test['d1_diasbp_invnoninv_max_diff'] = test['d1_diasbp_invasive_max'] - test['d1_diasbp_noninvasive_max']
test['h1_diasbp_invnoninv_max_diff'] = test['h1_diasbp_invasive_max'] - test['h1_diasbp_noninvasive_max']
test['d1_diasbp_invnoninv_min_diff'] = test['d1_diasbp_invasive_min'] - test['d1_diasbp_noninvasive_min']
test['h1_diasbp_invnoninv_min_diff'] = test['h1_diasbp_invasive_min'] - test['h1_diasbp_noninvasive_min']

test['d1_sysbp_invnoninv_max_diff'] = test['d1_sysbp_invasive_max'] - test['d1_sysbp_noninvasive_max']
test['h1_sysbp_invnoninv_max_diff'] = test['h1_sysbp_invasive_max'] - test['h1_sysbp_noninvasive_max']
test['d1_sysbp_invnoninv_min_diff'] = test['d1_sysbp_invasive_min'] - test['d1_sysbp_noninvasive_min']
test['h1_sysbp_invnoninv_min_diff'] = test['h1_sysbp_invasive_min'] - test['h1_sysbp_noninvasive_min']


for v in ['albumin','bilirubin','bun','glucose','hematocrit','pao2fio2ratio','arterial_ph','resprate','sodium','temp','wbc','creatinine']:
    train[f'{v}_indicator'] = (((train[f'{v}_apache']==train[f'd1_{v}_max']) & (train[f'd1_{v}_max']==train[f'h1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_max']) & (train[f'd1_{v}_max']==train[f'd1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_max']) & (train[f'd1_{v}_max']==train[f'h1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_max']) & (train[f'h1_{v}_max']==train[f'd1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_max']) & (train[f'h1_{v}_max']==train[f'h1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_max']) & (train[f'h1_{v}_max']==train[f'd1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_min']) & (train[f'd1_{v}_min']==train[f'd1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_min']) & (train[f'd1_{v}_min']==train[f'h1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_min']) & (train[f'd1_{v}_min']==train[f'h1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_min']) & (train[f'h1_{v}_min']==train[f'h1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_min']) & (train[f'h1_{v}_min']==train[f'd1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_min']) & (train[f'h1_{v}_min']==train[f'd1_{v}_max'])) 
                ).astype(np.int8)
    test[f'{v}_indicator'] = (((test[f'{v}_apache']==test[f'd1_{v}_max']) & (test[f'd1_{v}_max']==test[f'h1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_max']) & (test[f'd1_{v}_max']==test[f'd1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_max']) & (test[f'd1_{v}_max']==test[f'h1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_max']) & (test[f'h1_{v}_max']==test[f'd1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_max']) & (test[f'h1_{v}_max']==test[f'h1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_max']) & (test[f'h1_{v}_max']==test[f'd1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_min']) & (test[f'd1_{v}_min']==test[f'd1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_min']) & (test[f'd1_{v}_min']==test[f'h1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_min']) & (test[f'd1_{v}_min']==test[f'h1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_min']) & (test[f'h1_{v}_min']==test[f'h1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_min']) & (test[f'h1_{v}_min']==test[f'd1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_min']) & (test[f'h1_{v}_min']==test[f'd1_{v}_max'])) 
                ).astype(np.int8)

In [20]:
more_extreme_cols = [c for c in train.columns if(c.endswith("_day_more_extreme"))]

train["total_day_more_extreme"] = train[more_extreme_cols].sum(axis=1)
test["total_day_more_extreme"] = test[more_extreme_cols].sum(axis=1)

In [21]:
train[first_h].describe()


,wbc_started_after_firstHour,wbc_started_after_firstHour,creatinine_started_after_firstHour,creatinine_started_after_firstHour,sysbp_invasive_started_after_firstHour,sysbp_invasive_started_after_firstHour,hematocrit_started_after_firstHour,hematocrit_started_after_firstHour,arterial_ph_started_after_firstHour,arterial_ph_started_after_firstHour,...,platelets_started_after_firstHour,platelets_started_after_firstHour,calcium_started_after_firstHour,calcium_started_after_firstHour,bun_started_after_firstHour,bun_started_after_firstHour,hco3_started_after_firstHour,hco3_started_after_firstHour,albumin_started_after_firstHour,albumin_started_after_firstHour
count,130157,130157,130157,130157,130157,130157,130157,130157,130157,130157,...,130157,130157,130157,130157,130157,130157,130157,130157,130157,130157
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,False,False,True,True,False,False,...,True,True,True,True,True,True,True,True,False,False
freq,88557,88557,91511,91511,120388,120388,87368,87368,107115,107115,...,87172,87172,89229,89229,91256,91256,86348,86348,82558,82558


In [22]:
train["d1_resprate_div_mbp_min"] = train["d1_resprate_min"].div(train["d1_mbp_min"])
train["d1_resprate_div_sysbp_min"] = train["d1_resprate_min"].div(train["d1_sysbp_min"])
train["d1_lactate_min_div_diasbp_min"] = train["d1_lactate_min"].div(train["d1_diasbp_min"])
train["d1_heartrate_min_div_d1_sysbp_min"] = train["d1_heartrate_min"].div(train["d1_sysbp_min"])
train["d1_hco3_div"]= train["d1_hco3_max"].div(train["d1_hco3_min"])
train["d1_resprate_times_resprate"] = train["d1_resprate_min"].multiply(train["d1_resprate_max"])
train["left_average_spo2"] = (2*train["d1_spo2_max"] + train["d1_spo2_min"])/3

test["d1_resprate_div_mbp_min"] = test["d1_resprate_min"].div(test["d1_mbp_min"])
test["d1_resprate_div_sysbp_min"] = test["d1_resprate_min"].div(test["d1_sysbp_min"])
test["d1_lactate_min_div_diasbp_min"] = test["d1_lactate_min"].div(test["d1_diasbp_min"])
test["d1_heartrate_min_div_d1_sysbp_min"] = test["d1_heartrate_min"].div(test["d1_sysbp_min"])
test["d1_hco3_div"]= test["d1_hco3_max"].div(test["d1_hco3_min"])
test["d1_resprate_times_resprate"] = test["d1_resprate_min"].multiply(test["d1_resprate_max"])
test["left_average_spo2"] = (2*test["d1_spo2_max"] + test["d1_spo2_min"])/3

In [23]:
# sum of chronic diseases or complicators  - we may want to ignore the cancers/tumors +- immunosuppresasnts cancers or seperate
train["total_chronic"] = train[["aids","cirrhosis", 'hepatic_failure']].sum(axis=1)
train["total_cancer_immuno"] = train[[ 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].sum(axis=1)

# sum of chronic diseases or complicators  - we may want to ignore the cancers/tumors +- immunosuppresasnts cancers or seperate
test["total_chronic"] = test[["aids","cirrhosis", 'hepatic_failure']].sum(axis=1)
test["total_cancer_immuno"] = test[[ 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].sum(axis=1)

In [24]:
train["has_complicator"] = train[["aids","cirrhosis", 'hepatic_failure',
                            'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].max(axis=1)
test["has_complicator"] = test[["aids","cirrhosis", 'hepatic_failure',
                            'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].max(axis=1)
train[["has_complicator","total_chronic","total_cancer_immuno","has_complicator"]].describe()

,has_complicator,total_chronic,total_cancer_immuno,has_complicator
count,130157.000000,130157.000000,130157.000000,130157.000000
mean,0.069040,0.030709,0.058015,0.069040
std,0.253523,0.214531,0.275306,0.253523
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,1.000000,3.000000,4.000000,1.000000


In [25]:
train["int_apache_3j_diag"] =  train["apache_3j_diagnosis"].fillna(0).astype(str).str.split(".",expand=True)[0].astype(int)
test["int_apache_3j_diag"] =  test["apache_3j_diagnosis"].fillna(0).astype(str).str.split(".",expand=True)[0].astype(int)

In [26]:
import math
def myround(x, base=5):
    """https://stackoverflow.com/questions/2272149/round-to-5-or-other-number-in-python"""   
    if math.isnan(x): return (np.nan) # handle missing values
    else:
        return int(base * round(x/base))

train["rounded_age"] = train['age'].apply(lambda x: myround(x,base=8))
train["rounded_height"] = train['height'].apply(lambda x: myround(x,base=3))
test["rounded_age"] = test['age'].apply(lambda x: myround(x,base=8))
test["rounded_height"] = test['height'].apply(lambda x: myround(x,base=3))

IDENTIFYING_COLS = ['rounded_age', 'rounded_height',  'ethnicity', 'gender' , "aids", 'bmi'] 

In [27]:
train["hash_person_profile"] = train[IDENTIFYING_COLS].apply(lambda x: hash(tuple(x)), axis = 1)
test["hash_person_profile"] = test[IDENTIFYING_COLS].apply(lambda x: hash(tuple(x)), axis = 1)

print(train["hash_person_profile"].nunique())
display(train[["hash_person_profile",'rounded_age', 'rounded_height','age','height']].tail(2))

train.drop(['rounded_age', 'rounded_height'],axis=1,inplace=True)
test.drop(['rounded_age', 'rounded_height'],axis=1,inplace=True)

103457


,hash_person_profile,rounded_age,rounded_height,age,height
130155,-6562534046133179375,80.0,186.0,81.0,185.4
130156,-4286762246913857347,32.0,183.0,35.0,182.9


In [28]:
train["grouped_apache_3j_diag"] = train['int_apache_3j_diag'].fillna(0).astype(int).apply(lambda x: round(x,-2))//100
test["grouped_apache_3j_diag"] = test['int_apache_3j_diag'].fillna(0).astype(int).apply(lambda x: round(x,-2))//100

In [29]:
train['apache_3j'] = np.where(train['int_apache_3j_diag'].isna() , np.nan ,
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 200, 'Cardiovascular' , 
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 400, 'Respiratory' , 
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 500, 'Neurological' , 
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 600, 'Sepsis' , 
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 800, 'Trauma' ,  
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 900, 'Haematological' ,         
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 1000, 'Renal/Genitourinary' ,         
                            np.where(train['int_apache_3j_diag'].fillna(9999).astype('int') < 1200, 'Musculoskeletal/Skin disease' , 'Operative Sub-Diagnosis Codes' ))))))))
                                    )
test['apache_3j'] = np.where(test['int_apache_3j_diag'].isna() , np.nan ,
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 200, 'Cardiovascular' , 
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 400, 'Respiratory' , 
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 500, 'Neurological' , 
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 600, 'Sepsis' , 
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 800, 'Trauma' ,  
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 900, 'Haematological' ,         
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 1000, 'Renal/Genitourinary' ,         
                            np.where(test['int_apache_3j_diag'].fillna(9999).astype('int') < 1200, 'Musculoskeletal/Skin disease' , 'Operative Sub-Diagnosis Codes' ))))))))
                                    )

train['apache_3j_diagnosis_split1'] = np.where(train['apache_3j_diagnosis'].isna() , np.nan , train['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[1]  )
test['apache_3j_diagnosis_split1']  = np.where(test['apache_3j_diagnosis'].isna() , np.nan , test['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[1]  )
train['pre_icu_los_days'] = train['pre_icu_los_days'].apply(lambda x:special.expit(x) )
test['pre_icu_los_days']  = test['pre_icu_los_days'].apply(lambda x:special.expit(x) )
train['abmi'] = train['age']/train['bmi']
train['agi'] = train['weight']/train['age']
test['abmi'] = test['age']/train['bmi']
test['agi'] = test['weight']/train['age']

In [30]:
# https://diatribe.org/understanding-average-glucose-standard-deviation-cv-and-blood-sugar-variability
#train['std_glucose']=train['glucose_apache'].std()
#test['std_glucose']=test['glucose_apache'].std()
train['std_glucose']=train['d1_glucose_max']+ train['d1_glucose_min'].std()
test['std_glucose']=test['d1_glucose_max']+ train['d1_glucose_min'].std()
train['std1_glucose']=train['glucose_apache']- train['d1_glucose_min'].std()
test['std1_glucose']=test['glucose_apache']- train['d1_glucose_min'].std()
train['std2_glucose']=train['glucose_apache']- train['d1_glucose_max'].std()
test['std2_glucose']=test['glucose_apache']- train['d1_glucose_max'].std()
#https://bmcnephrol.biomedcentral.com/articles/10.1186/s12882-019-1381-3
train['urineoutput_std'] = train['urineoutput_apache'].std()
test['urineoutput_std'] = test['urineoutput_apache'].std()
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4267581/
#train['gcs_function_std'] = train['gcs_function'].std()
#test['gcs_function_std'] = test['gcs_function'].std()
train['gcs_function_std'] = train['gcs_eyes_apache']+train['gcs_motor_apache']+train['gcs_verbal_apache'].std()
test['gcs_function_std'] = test['gcs_eyes_apache']+train['gcs_motor_apache']+train['gcs_verbal_apache'].std()
#https://www.mdedge.com/emergencymedicine/article/86484/endocrinology/diabetes-mellitus-diagnosis-and-management-emergency
train['diagnosis_dm'] = train['ethnicity']+train['hospital_admit_source'].std()
train['diagnosis_dm_mean'] = train['ethnicity']+train['hospital_admit_source'].mean()
test['diagnosis_dm'] = test['ethnicity']+test['hospital_admit_source'].std()
test['diagnosis_dm_mean'] = test['ethnicity']+test['hospital_admit_source'].mean()


In [31]:
train = train.replace([np.inf, -np.inf], np.nan)
train= train.fillna(0)


In [32]:
test = test.replace([np.inf, -np.inf], np.nan)
test= test.fillna(0)


In [33]:
trainf = pd.read_pickle('../input/widsfeatures2021/X.pkl')
testf = pd.read_pickle('../input/widsfeatures2021/X_test.pkl')

In [34]:

train.shape, test.shape



((130157, 691), (10234, 690))

In [35]:
train = pd.concat([trainf.reset_index(drop=True),train.reset_index(drop=True)], axis=1)
test =  pd.concat([testf.reset_index(drop=True),test.reset_index(drop=True)], axis=1)
train.shape, test.shape

((130157, 1556), (10234, 1555))

In [36]:
TGT_COL = "diabetes_mellitus"
y = train[TGT_COL].values
del train[TGT_COL]

In [37]:
test = test.replace([np.inf, -np.inf], np.nan)
test= test.fillna(0)

train = train.replace([np.inf, -np.inf], np.nan)
train= train.fillna(0)

In [38]:
Cols = list(train.columns)
for i,item in enumerate(train.columns):
    if item in train.columns[:i]: Cols[i] = "toDROP"
train.columns = Cols
test.columns = Cols
train = train.drop("toDROP",1)
test = test.drop("toDROP",1)
train.shape, test.shape

((130157, 1101), (10234, 1101))

In [39]:
@contextmanager
def timer(logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None):
    if prefix: format_str = str(prefix) + format_str
    if suffix: format_str = format_str + str(suffix)
    start = time.time()
    yield
    d = time.time() - start
    out_str = format_str.format(d)
    if logger:
        logger.info(out_str)
    else:
        print(out_str)

In [40]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelEncoder, OrdinalEncoder

cats = [ 
 'elective_surgery',
 'icu_id',
 'icu_stay_type',
 'apache_post_operative',
 'arf_apache',
 'intubated_apache',
 'ventilated_apache',
 'gcs_unable_apache',
 'cirrhosis',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis',
    'apache_3j_diagnosis_x',
    'apache_2_diagnosis_x',
    'apache_3j','apache_3j_diagnosis_split1','cat_gcs_function'
    
]


for col in cats:
    print(col)
    train_only = list(set(train[col].unique()) - set(test[col].unique()))
    test_only = list(set(test[col].unique()) - set(train[col].unique()))
    both = list(set(test[col].unique()).union(set(train[col].unique())))
    print(len(both), len(train_only), len(test_only))
    train.loc[train[col].isin(train_only), col] = np.nan
    test.loc[test[col].isin(test_only), col] = np.nan
    try:
        lbl = OrdinalEncoder(dtype='int')
        train[col] = lbl.fit_transform(train[col].astype('str').values.reshape(-1,1))
        test[col] = lbl.transform(test[col].astype('str').values.reshape(-1,1))
        temp = pd.concat([train[[col]], test[[col]]], axis=0)
        temp_mapping = temp.groupby(col).size()/len(temp)
        temp['enc'] = temp[col].map(temp_mapping)
        temp['enc'] = stats.rankdata(temp['enc'])
        temp = temp.reset_index(drop=True)
        train[f'rank_frqenc_{col}'] = temp[['enc']].values[:train.shape[0]]
        test[f'rank_frqenc_{col}'] = temp[['enc']].values[train.shape[0]:]               
        test[col] = test[col].astype('category')
        train[col] = train[col].astype('category')
    except:
        try:
            lbl = LabelEncoder()
            fit_by = pd.Series([i for i in X[col].unique() if type(i) == str])
            lbl.fit(fit_by)
            train[col] = train[col].apply(lambda x: lbl.transform([x])[0] if type(x) == str else x)
            test[col] = test[col].apply(lambda x: lbl.transform([x])[0] if type(x) == str else x)
        except:
            drop_columns.append(col)

elective_surgery
2 0 0
icu_id
366 64 38
icu_stay_type
3 0 0
apache_post_operative
2 0 0
arf_apache
2 0 0
intubated_apache
2 0 0
ventilated_apache
2 0 0
gcs_unable_apache
3 0 0
cirrhosis
2 0 0
hepatic_failure
2 0 0
immunosuppression
2 0 0
leukemia
2 0 0
lymphoma
2 0 0
solid_tumor_with_metastasis
2 0 0
apache_3j_diagnosis_x
112 1 0
apache_2_diagnosis_x
45 0 0
apache_3j
9 0 0
apache_3j_diagnosis_split1
26 0 0
cat_gcs_function
7 0 0


In [41]:
train

,intubated_apache,h1_platelets_min,h1_bilirubin_max,glucose_wb,h1_hematocrit_max,d1_arterial_po2_max,d1_mbp_invasive_max,gcs_function,cat_bmi,d1_mbp_invasive_min,...,rank_frqenc_cirrhosis,rank_frqenc_hepatic_failure,rank_frqenc_immunosuppression,rank_frqenc_leukemia,rank_frqenc_lymphoma,rank_frqenc_solid_tumor_with_metastasis,rank_frqenc_apache_3j_diagnosis_x,rank_frqenc_apache_2_diagnosis_x,rank_frqenc_apache_3j,rank_frqenc_apache_3j_diagnosis_split1
0,0,0.0,0.0,0,0.0,0.0,66.0,3,2,40.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,86995.0,131638.0,44895.0,98782.5
1,0,557.0,0.0,0,36.9,51.0,0.0,8,3,0.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,41472.0,14065.0,66071.0,98782.5
2,0,0.0,0.0,0,0.0,0.0,0.0,4,4,0.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,106093.0,75168.0,27750.0,35652.5
3,1,43.0,0.0,4,34.0,337.0,92.0,5,2,52.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,68972.5,55154.5,93096.0,35652.5
4,0,0.0,0.0,0,0.0,0.0,0.0,13,0,0.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,62856.5,30360.5,27750.0,98782.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130152,0,0.0,0.0,0,0.0,0.0,0.0,5,3,0.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,91201.5,59592.5,124045.0,98782.5
130153,0,0.0,0.0,0,0.0,0.0,0.0,4,3,0.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,33465.5,10903.5,66071.0,98782.5
130154,1,232.0,0.5,4,33.0,163.0,110.0,6,4,59.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,75550.5,40047.5,124045.0,98782.5
130155,0,0.0,0.0,0,33.0,0.0,0.0,5,2,0.0,...,71304.0,71132.5,71982.0,70705.0,70489.5,71643.5,43281.0,86692.5,66071.0,98782.5


In [42]:
class TreeModel:
    """Wrapper for LightGBM/XGBoost/CATBoost"""
    def __init__(self, model_type: str):
        self.model_type = model_type
        self.trn_data = None
        self.val_data = None
        self.model = None

    def train(self,
              params: dict,
              X_train: pd.DataFrame, y_train: np.ndarray,
              X_val: pd.DataFrame, y_val: np.ndarray,
              train_weight: tp.Optional[np.ndarray] = None,
              val_weight: tp.Optional[np.ndarray] = None,
              train_params: dict = {}):
        if self.model_type == "lgb":
            self.trn_data = lgb.Dataset(X_train, label=y_train, weight=train_weight)
            self.val_data = lgb.Dataset(X_val, label=y_val, weight=val_weight)
            self.model = lgb.train(params=params,
                                   train_set=self.trn_data,
                                   valid_sets=[self.trn_data, self.val_data],
                                   **train_params)
        elif self.model_type == "xgb":
            self.trn_data = xgb.DMatrix(X_train, y_train, weight=train_weight)
            self.val_data = xgb.DMatrix(X_val, y_val, weight=val_weight)
            self.model = xgb.train(params=params,
                                   dtrain=self.trn_data,
                                   evals=[(self.trn_data, "train"), (self.val_data, "val")],
                                   **train_params)
        elif self.model_type == "cat":
            self.trn_data = Pool(X_train, label=y_train, group_id=[0] * len(X_train))
            self.val_data =  Pool(X_val, label=y_val, group_id=[0] * len(X_val))
            self.model = CatBoost(params)
            self.model.fit(
                self.trn_data, eval_set=[self.val_data], use_best_model=True, **train_params)
        else:
            raise NotImplementedError

    def predict(self, X: pd.DataFrame):
        if self.model_type == "lgb":
            return self.model.predict(
                X, num_iteration=self.model.best_iteration)  # type: ignore
        elif self.model_type == "xgb":
            X_DM = xgb.DMatrix(X)
            return self.model.predict(
                X_DM, ntree_limit=self.model.best_ntree_limit)  # type: ignore
        elif self.model_type == "cat":
            return self.model.predict(X)
        else:
            raise NotImplementedError

    @property
    def feature_names_(self):
        if self.model_type == "lgb":
            return self.model.feature_name()
        elif self.model_type == "xgb":
            return list(self.model.get_score(importance_type="gain").keys())
        elif self.model_type == "cat":
             return self.model.feature_names_
        else:
            raise NotImplementedError

    @property
    def feature_importances_(self):
        if self.model_type == "lgb":
            return self.model.feature_importance(importance_type="gain")
        elif self.model_type == "xgb":
            return list(self.model.get_score(importance_type="gain").values())
        elif self.model_type == "cat":
            return self.model.feature_importances_
        else:
            raise NotImplementedError

In [43]:
numeric_type = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
cat_type = ['category','object']
numeric_cols = train[selector(dtype_include=numeric_type)].columns
nums = numeric_cols
drop_columns=[]
high_corr = []
for num_0 in nums:
    for num_1 in nums:
        if (num_0, num_1) not in high_corr and (num_1, num_0) not in high_corr and num_0 != num_1:
            corr_ = np.corrcoef(train[num_0], train[num_1])[0][1]
            if corr_ >=0.9998 or corr_<=-0.9998:
                high_corr.append((num_0, num_1))
                high_corr.append((num_1, num_0))
                print(f'FEAT_A: {num_0} FEAT_B: {num_1} - Correlation: {corr_}')
                drop_columns.append(num_0)

FEAT_A: h1_bilirubin_max FEAT_B: h1_bilirubin_min - Correlation: 0.9999658415700183
FEAT_A: h1_sodium_max FEAT_B: h1_sodium_min - Correlation: 0.9999582938287919
FEAT_A: paco2_apache FEAT_B: paco2_for_ph_apache - Correlation: 1.0
FEAT_A: h1_bun_max FEAT_B: h1_bun_min - Correlation: 0.9998662997685427
FEAT_A: h1_albumin_max FEAT_B: h1_albumin_min - Correlation: 0.999950402561671
FEAT_A: d1_arterial_ph_max FEAT_B: d1_arterial_ph_min - Correlation: 0.9998822366038396
FEAT_A: h1_arterial_ph_min FEAT_B: h1_arterial_ph_max - Correlation: 0.9999836824666547
FEAT_A: d1_inr_min FEAT_B: h1_inr_min - Correlation: 1.0
FEAT_A: d1_inr_max FEAT_B: h1_inr_max - Correlation: 1.0
FEAT_A: arf_apache_std_d1_hemaglobin_max FEAT_B: rank_frqenc_arf_apache - Correlation: 0.9999999999999972
FEAT_A: mean_diff_d1_inr_min FEAT_B: mean_diff_h1_inr_min - Correlation: 1.0
FEAT_A: ventilated_apache_std_d1_glucose_min FEAT_B: rank_frqenc_ventilated_apache - Correlation: -1.0
FEAT_A: rank_frqenc_apache_2_diagnosis FEAT

In [44]:
for col in test.columns:
    r = ks_2samp(train[col].dropna(), test[col].dropna())
    if r[0] >= 0.05:

        print(col, r)
        drop_columns.append(col)


cat_bmi Ks_2sampResult(statistic=0.0646824065479082, pvalue=6.631024544701579e-35)
weight Ks_2sampResult(statistic=0.06749177507876722, pvalue=5.773329346233341e-38)
bmi Ks_2sampResult(statistic=0.0646824065479082, pvalue=6.631024544701579e-35)
cat_bmi_std_bmi Ks_2sampResult(statistic=0.0646824065479082, pvalue=6.631024544701579e-35)
cat_bmi_mean_d1_wbc_min Ks_2sampResult(statistic=0.0646824065479082, pvalue=6.631024544701579e-35)
cat_bmi_std_d1_resprate_min Ks_2sampResult(statistic=0.06468240654790824, pvalue=6.631024544700825e-35)
cat_bmi_std_d1_sodium_min Ks_2sampResult(statistic=0.05436559937883169, pvalue=8.77660001217563e-25)
cat_bmi-glucose_rate Ks_2sampResult(statistic=0.06477460289539624, pvalue=5.28711374909374e-35)
cat_bmi_std_d1_bun_min Ks_2sampResult(statistic=0.054275441278717085, pvalue=1.0569326836601183e-24)
rank_frqenc_cat_bmi Ks_2sampResult(statistic=0.0646824065479082, pvalue=6.631024544701579e-35)
cat_bmi-cat_age Ks_2sampResult(statistic=0.06917880052344716, pvalue

In [45]:
drop_columns

['h1_bilirubin_max',
 'h1_sodium_max',
 'paco2_apache',
 'h1_bun_max',
 'h1_albumin_max',
 'd1_arterial_ph_max',
 'h1_arterial_ph_min',
 'd1_inr_min',
 'd1_inr_max',
 'arf_apache_std_d1_hemaglobin_max',
 'mean_diff_d1_inr_min',
 'ventilated_apache_std_d1_glucose_min',
 'rank_frqenc_apache_2_diagnosis',
 'apache_post_operative_std_d1_temp_max',
 'intubated_apache_mean_d1_spo2_max',
 'mean_diff_h1_bilirubin_min',
 'mean_diff_h1_inr_max',
 'elective_surgery_mean_d1_sysbp_min',
 'inr_d1_value_range',
 'cat_bmi',
 'weight',
 'bmi',
 'cat_bmi_std_bmi',
 'cat_bmi_mean_d1_wbc_min',
 'cat_bmi_std_d1_resprate_min',
 'cat_bmi_std_d1_sodium_min',
 'cat_bmi-glucose_rate',
 'cat_bmi_std_d1_bun_min',
 'rank_frqenc_cat_bmi',
 'cat_bmi-cat_age',
 'mean_diff_d1_calcium_min',
 'cat_bmi_mean_d1_sodium_min',
 'cat_bmi_mean_d1_temp_max',
 'cat_bmi-gender',
 'cat_gcs_function-apache_3j_diagnosis',
 'cat_bmi-icu_stay_type',
 'intubated_apache-cat_bmi',
 'cat_bmi-gcs_verbal_apache',
 'cat_bmi-icu_admit_source'

In [46]:
drop_columns_ks05 = ['paco2_apache',
 'd1_inr_max',
 'd1_inr_min',
 'h1_albumin_max',
 'h1_bilirubin_max',
 'h1_bun_max',
 'h1_sodium_max',
 'd1_arterial_ph_max',
 'h1_arterial_ph_max',
 'inr_d1_value_range',
 'mean_diff_d1_inr_max',
 'mean_diff_d1_inr_min',
 'mean_diff_h1_bilirubin_max',
 'bmi',
 'weight',
 'cat_bmi',
 'icu_id_counts',
 'diff_bmi',
 'spo2_h1_value_range',
 'spo2_tot_change_value_range_normed',
 'mean_diff_d1_calcium_max',
 'mean_diff_d1_calcium_min',
 'abmi',
 'agi']
drop_columns_ks10 = ['paco2_apache',
 'd1_inr_max',
 'd1_inr_min',
 'h1_albumin_max',
 'h1_bilirubin_max',
 'h1_bun_max',
 'h1_sodium_max',
 'd1_arterial_ph_max',
 'h1_arterial_ph_max',
 'inr_d1_value_range',
 'mean_diff_d1_inr_max',
 'mean_diff_d1_inr_min',
 'mean_diff_h1_bilirubin_max','diff_bmi', 'abmi'
]

# Training 


In [47]:
drops = set(['Unnamed: 0',
 'aids',
 'albumin_apache',
 'apache_post_operative',
 'cat_age',
 'cat_bmi',
 'd1_albumin_min',
 'd1_arterial_pco2_min',
 'd1_arterial_ph_min',
 'd1_calcium_max',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_hematocrit_min',
 'd1_inr_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_pao2fio2ratio_max',
 'd1_pao2fio2ratio_min',
 'd1_platelets_max',
 'd1_resprate_max',
 'd1_sysbp_invasive_min',
 'd1_temp_min',
 'd1_wbc_min',
 'gcs_unable_apache',
 'h1_albumin_max',
 'h1_albumin_min',
 'h1_arterial_pco2_max',
 'h1_arterial_pco2_min',
 'h1_arterial_po2_max',
 'h1_bilirubin_max',
 'h1_bun_max',
 'h1_creatinine_min',
 'h1_diasbp_noninvasive_max',
 'h1_heartrate_max',
 'h1_heartrate_min',
 'h1_hemaglobin_min',
 'h1_hematocrit_max',
 'h1_hematocrit_min',
 'h1_lactate_max',
 'h1_lactate_min',
 'h1_mbp_invasive_max',
 'h1_mbp_invasive_min',
 'h1_mbp_max',
 'h1_mbp_min',
 'h1_mbp_noninvasive_max',
 'h1_mbp_noninvasive_min',
 'h1_pao2fio2ratio_max',
 'h1_pao2fio2ratio_min',
 'h1_platelets_max',
 'h1_platelets_min',
 'h1_resprate_max',
 'h1_resprate_min',
 'h1_spo2_max',
 'h1_spo2_min',
 'h1_sysbp_max',
 'h1_sysbp_min',
 'h1_sysbp_noninvasive_max',
 'h1_sysbp_noninvasive_min',
 'h1_temp_max',
 'h1_temp_min',
 'h1_wbc_max',
 'h1_wbc_min',
 'height',
 'hematocrit_apache',
 'lymphoma',
 'map_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'resprate_apache','hash_person_profile', 
 'wbc_apache','albumin_h1_value_range','albumin_tot_change_value_range_normed','bilirubin_h1_value_range','bilirubin_tot_change_value_range_normed',
'inr_day_more_extreme','inr_started_after_firstHour','bun_h1_value_range']+drop_columns_ks10 )

In [48]:
len(drops)

94

In [49]:
FEAT_COLS =  list(set(test.columns.tolist()) - drops)

In [50]:
len(FEAT_COLS)

1025

In [51]:
train[FEAT_COLS].to_pickle('train.pkl')
test[FEAT_COLS].to_pickle('test.pkl')
np.save('test_id.npy',test_id)
np.save('y.npy',y)

In [52]:
ID_COL = "encounter_id"
#TGT_COL = "diabetes_mellitus"
# https://towardsdatascience.com/hyperparameter-tuning-to-reduce-overfitting-lightgbm-5eb81a0b464e

N_SPLITS =10# 5
RANDOM_SEED = 190295 #,42,90295#42
USE_MODEL = "lgb"

MODEL_PARAMS = {
   "seed": RANDOM_SEED,
      "n_jobs": 4,
       'num_leaves': 250,
    'max_bin':550,
     'min_child_weight': 0.03454472573214212,
    'feature_fraction': 0.35, #0.52 
    'bagging_fraction': 0.475, #0.52 0.8
    'min_data_in_leaf': 125, #150,
     'max_depth': -1,
    'objective': 'binary',
    'learning_rate': 0.01,
    "boosting_type": "gbdt",
    "metric": 'auc', 
     "bagging_seed": 111,#7,
#"verbosity": -1,
'reg_alpha': 0.3899927210061127,
'reg_lambda': 0.6485237330340494,
    
      ##
'cat_smooth': 39,
 ## parameters to keep the exactly the same
'subsample_for_bin': 200000,
'min_child_samples':285, #20,    
    
    
    
    }
TRAIN_PARAMS = {
    "num_boost_round": 30000,
    "early_stopping_rounds": 500,
    "verbose_eval": 50,
}

In [53]:
train[FEAT_COLS]

,gcs_function_mean_h1_diasbp_min,arterial_ph_apache,apache_2_diagnosis,mean_diff_h1_glucose_min,glucose_h1_zero_range,d1_glucose_min,hco3_tot_change_value_range_normed,gcs_motor_apache_mean_h1_diasbp_min,wbc_d1_value_range,glucose_d1_zero_range,...,apache_3j_diagnosis_mean_d1_platelets_min,pao2fio2ratio_h1_zero_range,diasbp_tot_change_value_range_normed,mean_diff_d1_bilirubin_max,rank_frqenc_gcs_verbal_apache,rank_frqenc_glucose_rate,mean_diff_d1_hco3_max,hemaglobin_day_more_extreme,icu_type-gcs_motor_apache,h1_hemaglobin_max
0,61.618537,0.000,11,0.000000,0,109.0,0.0,63.045995,0.0,0,...,185.989968,0,6.200000,-0.708935,18572.0,99845.0,-3.031935,False,41,0.0
1,61.217593,7.450,7,-18.696927,0,128.0,0.0,61.546968,10.6,0,...,197.734779,1,4.923077,-0.785212,39347.5,29864.0,1.070795,True,3,11.3
2,61.894083,0.000,20,0.000000,0,0.0,0.0,63.045995,0.0,0,...,214.588091,0,1.333333,0.000000,96006.0,99845.0,0.000000,False,6,0.0
3,63.635914,7.390,24,0.000000,0,88.0,0.0,63.045995,1.0,0,...,124.946802,1,0.333333,0.000000,96006.0,99845.0,0.000000,True,41,11.6
4,59.853901,0.000,17,0.000000,0,0.0,0.0,59.853901,0.0,0,...,195.134671,0,1.354839,0.000000,1198.0,99845.0,0.000000,False,7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130152,63.635914,0.000,9,0.000000,0,0.0,0.0,63.045995,0.0,0,...,203.498827,0,1.000000,0.000000,96006.0,99845.0,0.000000,False,34,0.0
130153,61.894083,0.000,4,0.000000,0,139.0,0.0,63.045995,0.9,1,...,198.505110,0,17.000000,-0.109127,18572.0,99845.0,2.330880,False,20,0.0
130154,58.860403,7.224,12,0.000000,0,55.0,0.0,58.974454,21.4,0,...,185.518591,1,0.000000,-0.583642,39347.5,99845.0,2.802911,True,29,10.2
130155,63.635914,0.000,22,0.000000,0,137.0,0.0,63.045995,0.0,0,...,177.278183,0,2.000000,0.000000,96006.0,99845.0,-3.954759,True,6,10.9


In [54]:
X = train[FEAT_COLS]
X_test = test[FEAT_COLS]


kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
trn_val_indexs = list(kf.split(X, y))

In [55]:
oof_pred_arr = np.zeros(len(X))
test_preds_arr = np.zeros((N_SPLITS, len(X_test)))
feature_importances = pd.DataFrame()
score_list = []

In [56]:
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print("*" * 100)
    print(f"Fold: {fold}")

    X_trn = X.loc[trn_idx].reset_index(drop=True)
    X_val = X.loc[val_idx].reset_index(drop=True)
    y_trn = y[trn_idx]
    y_val = y[val_idx]

    model = TreeModel(model_type=USE_MODEL)
    with timer(prefix="Model training"):
        model.train(
            params=MODEL_PARAMS, X_train=X_trn, y_train=y_trn,
            X_val=X_val, y_val=y_val, train_params=TRAIN_PARAMS)
    fi_tmp = pd.DataFrame()
    fi_tmp["feature"] = model.feature_names_
    fi_tmp["importance"] = model.feature_importances_
    fi_tmp["fold"] = fold
    feature_importances = feature_importances.append(fi_tmp)

    val_pred = model.predict(X_val)
    score = roc_auc_score(y_val, val_pred)#, squared=False)
    #print(score)
    print(f"score: {score:.5f}")
    score_list.append([fold, score])
    oof_pred_arr[val_idx] = val_pred
    test_pred = model.predict(X_test)
    test_preds_arr[fold] = test_pred

****************************************************************************************************
Fold: 0
[LightGBM] [Warning] min_data_in_leaf is set=125, min_child_samples=285 will be ignored. Current value: min_data_in_leaf=125
[LightGBM] [Warning] min_data_in_leaf is set=125, min_child_samples=285 will be ignored. Current value: min_data_in_leaf=125
[LightGBM] [Info] Number of positive: 25377, number of negative: 91764
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.310867 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 156423
[LightGBM] [Info] Number of data points in the train set: 117141, number of used features: 1023
[LightGBM] [Warning] min_data_in_leaf is set=125, min_child_samples=285 will be ignored. Current value: min_data_in_leaf=125
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.216636 -> initscore=-1.285377
[Lig

In [57]:
oof_score = roc_auc_score(y, oof_pred_arr)#, squared=False)
score_list.append(["oof", oof_score])
pd.DataFrame(
    score_list, columns=["fold", "rscore"])

,fold,rscore
0,0,0.880252
1,1,0.877754
2,2,0.880458
3,3,0.886134
4,4,0.877438
5,5,0.881345
6,6,0.885649
7,7,0.878725
8,8,0.877583
9,9,0.881247


In [58]:
test_preds_arr = test_preds_arr.max(axis=0)

In [59]:
#Create a  DataFrame
submission = pd.DataFrame({'encounter_id':test_id,'diabetes_mellitus':test_preds_arr})
                        

#Visualize the first 10 rows
submission.head(10)

,encounter_id,diabetes_mellitus
0,144740,0.068537
1,141990,0.339850
2,142038,0.136856
3,138628,0.067389
4,141682,0.379612
5,139096,0.043260
6,142994,0.817771
7,141954,0.057646
8,135344,0.726991
9,142552,0.877451


In [60]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as your notebook
filename = 'submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submission.csv


In [61]:
np.save('oof_preds.py',oof_pred_arr)
np.save('test_preds.py',test_preds_arr)